<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/English%E4%BA%8C%E6%AC%A1%E5%AF%B9%E8%AF%9D%E6%8A%BD%E5%8F%96%E6%95%B0%E6%8D%AE%E5%87%86%E5%A4%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [ ]:
import os
import httpx

os.environ["OPENAI_API_KEY"]  = "sk-JEBR"

import openai
from openai import AsyncOpenAI


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
input_name = "/content/drive/MyDrive/CardBuild/exp0122/english_segs.txt"

import json

seg_datas = []

with open(input_name, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip() == "":
            continue
        data = json.loads(line)
        seg_datas.append(data)


In [ ]:
print(seg_datas[0].keys())

dict_keys(['id', 'raw_seg'])


In [ ]:
print(seg_datas[0]['id'])

0


In [ ]:
print(len(seg_datas))

17613


In [ ]:
id2seg_index = {}
for i, data in enumerate(seg_datas):
    id2seg_index[str(data["id"])+"_first"] =  i

In [ ]:
print(seg_datas[0]['id'])

0


In [ ]:
save_path = "/content/drive/MyDrive/CardBuild/exp0122/first/"

datas = []

import json

# 遍历指定文件夹中的所有txt文件
for filename in os.listdir(save_path):
    if filename.endswith(".txt") and filename.startswith("0122english_"):
        # 构建完整的文件路径
        file_path = os.path.join(save_path, filename)

        # 打开并读取文件
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                data = json.loads(line)
                response_str = data["response"]
                try:
                    response = json.loads(response_str)
                    data["response"] = response
                    datas.append(data)
                except:
                    continue


In [ ]:
print(len(datas))

17593


In [ ]:
print(datas[250].keys())
print(datas[25]['id'])
# print(datas[25]["text"])
print(datas[25]["response"])

dict_keys(['id', 'text', 'n_quote', 'response'])
10233_first
{'summary': 'Edith suggests Julian visit the gymnasium to see how they train. The doctor explains the public training for all-around physical development. They discuss the differences in physical education between the 20th and 21st centuries. They observe a foot race and discuss the inclusivity of the youth in the event.', 'conversations': [{'num': 1, 'dialogue': 'Considering what you have been telling Julian about women nowadays as compared with the old days, I wonder if he would not be interested in visiting the gymnasium this afternoon and seeing something of how we train ourselves? There are going to be some foot races and air races, and a number of other tests. It is the afternoon when our year has the grounds, and I ought to be there anyway.', 'said by': 'Edith'}, {'num': 3, 'dialogue': 'To this suggestion, which was eagerly accepted, I owe one of the most interesting and instructive experiences of those early days duri

因为是第一次抽取，所以我们只要每个seg的第一个

In [ ]:
def count_quotes(text):

    quote_chars = '''「」"“”'''
    count = 0
    for char in text:
        if char in quote_chars:
            count += 1
    return count

input_datas = []

from tqdm import tqdm

for data in tqdm(datas):
    id = data['id']
    if id not in id2seg_index:
        print(id, ' not found')
        continue
    seg_index = id2seg_index[id]
    segs = seg_datas[seg_index]["raw_seg"]
    if len(segs) < 2:
        continue

    if "summary" not in data["response"]:
        continue

    text = data["response"]["summary"] + "\n" + segs[1]
    id_str = data["id"]
    if id_str.endswith("_first"):
        id_str = id_str.replace("_first", "_second")
    else:
        id_str = id_str + "_second"

    input_data = {
        "id": id_str,
        "text": text,
        "n_quote": count_quotes(text)
    }
    input_datas.append(input_data)
    # break

100%|██████████| 17593/17593 [00:03<00:00, 5825.69it/s]


In [ ]:
print(len(input_datas))

15804


In [ ]:
print(input_datas[0]['text'])

Cory Doctorow wrote Little Brother in a white-hot fury over the course of eight weeks. The book was trying to get out of his head, leading to sleep deprivation and missed meals. The author's father was one of the few 'counterculture' people who thought computers were a good thing in the 1960s. The author's fascination with activist causes grew as he saw how communications tools were being used to bring information and revolution to authoritarian states. However, 17 years later, computers are being used to spy, control, and restrict freedoms. The author expresses concern about the dangers of technology and the loss of privacy and freedom. The book is meant to be part of the conversation about what an information society means and encourages taking action for freedom and privacy.
I wrote Little Brother in a white-hot fury between May 7, 2007 and July 2, 2007: exactly eight weeks from the day I thought it up to the day I finished it (Alice, to whom this book is dedicated, had to put up wi

给定Paragraph，抽取其中的对话，并输出为json格式

Let's think it step by step
1. 对Paragraph进行总结，存储在summary字段
2. 抽取每一句对话的内容 dialogue，以及对话的说话人 said by, 存储在conversations中



In [ ]:
random_order = list(range(len(input_datas)))
import random
random.shuffle(random_order)

for id in random_order:
    data = input_datas[id]
    if data["n_quote"] > 30:
        print(data["text"])
        break

Chapter 1
It's been one week since Deuce has been born and Sarah and Damion are sitting in the hospital room alone and Sarah says,"Are you going to change me now that Deuce has been born?"  "No. I absolutely refuse to change you," Damion said.  "But what happens if they find us and the elder finds out that I had the baby and you haven't changed me yet?" Sarah asked.  "I-," Damion replied.  "Well? Are you going to answer me?" Sarah asked.  "I don't know what's going to happen. If they find us they arer going to want me dead," Damion replied.  "Why would they want you dead?" Sarah asked.  "Cause I killed Erika," Damion replied,"When are you going to take Deuce home anyway?"  "I don't know," Sarah said in a pissed off tone.  "Are you mad at me now?" Damion asked.  "What do you think?" Sarah replied,"Are you going to change me or let them kill me if they find us?"  "I would really hate to see you killed. But I also don't want this life for you," Damion said.  "I understand that. But I don'

In [ ]:
task_prompt = """"Given an input paragraph, extract the dialogues within it, and output them in JSON format.

Let's think about it step by step:
- Summarize the input paragraph into bullet points and store it in the 'summary' field.
- recall the line number('num'), Extract the content of each dialogue ('dialogue'), identify the speaker for each sentence ('said by'), and store these in 'conversations'."""


long_task_prompt_prefix = """Given an input paragraph, extract the dialogues within it, and output them in JSON format.

Let's think about it step by step:
- Summarize the input paragraph into bullet points and store it in the 'summary' field.
- recall the line number('num'), Extract the content of each dialogue ('dialogue'), identify the speaker for each sentence ('said by'), and store these in 'conversations'."""

no_dialogue_prompt_hint = "if it is not a dialogue, output conversation as an empty list"

long_task_prompt_example = """
Example input paragraph:
1 The sun was setting behind the hills, casting long shadows across the valley. Birds chirped their evening songs, and a gentle breeze stirred the leaves.
2 Amidst this serene scene, Alex and Jamie were having a heated argument. Alex exclaimed, 'I can't believe you didn't tell me about the meeting yesterday!' Jamie retorted, 'Well, I thought you already knew since it was on the calendar!'
3 Their voices rose above the tranquil sounds of nature. Alex, trying to calm down, said, 'We need to communicate better in the future.'
4 Jamie agreed, replying, 'You're right, I should have double-checked with you.'

example output:
{
    "summary": "Sunset behind hills, casting shadows in the valley. Birds chirping, gentle breeze stirring leaves. Alex and Jamie having a heated argument. Discussion about a missed meeting and communication issues.",
    "conversations": [
        {
            "num": 2,
            "dialogue": "I can't believe you didn't tell me about the meeting yesterday!",
            "said by": "Alex"
        },
        {
            "num": 2,
            "dialogue": "Well, I thought you already knew since it was on the calendar!",
            "said by": "Jamie"
        },
        {
            "num": 3,
            "dialogue": "We need to communicate better in the future.",
            "said by": "Alex"
        },
        {
            "num": 4,
            "dialogue": "You're right, I should have double-checked with you.",
            "said by": "Jamie"
        }
    ]
}
"""


In [ ]:
!pip install -q aiofiles tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
import os
import json
import time
# import openai
from openai import AsyncOpenAI

aclient = AsyncOpenAI()

import asyncio
import aiofiles
import tiktoken
import hashlib
# from connector import AsyncPGConnector
from tqdm.asyncio import tqdm as tqdm

enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
en2zh_ratio = 2.3

delay = 1
concurrency_limit = 16

max_file_size = 1024**3

In [ ]:
async def dealing_messages(messages):
    try:
        # request_token = sum([len(enc.encode(msg['content'])) for msg in messages])
        # response_token = int(len(enc.encode(text)) * en2zh_ratio) + 64

        model = "gpt-3.5-turbo-1106"

        resp = await aclient.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0,
            response_format={ "type": "json_object" }
        )

        try:
            result = resp.choices[0].message.content
            result = result.strip()
            return result
        except json.JSONDecodeError:
            print("Invalid json: ", result)
            return None
        except:
            raise Exception(f"Invalid API response: {resp}")

    except Exception as e:
        print(f"[Error] {e}")
        return None


def add_line_num( text ):
    new_text = ""
    lines = text.split("\n")
    line_count = 1
    for line in lines:
        if line.strip() == "":
            new_text += "\n"
        else:
            new_text += f"{line_count} {line}\n"
            line_count += 1
    return new_text

def data2messages( data ):
    n_quote = data["n_quote"]
    if n_quote >= 20:
        task_prompt = long_task_prompt_prefix + "\n" + long_task_prompt_example
    else:
        task_prompt = long_task_prompt_prefix + "\n" + no_dialogue_prompt_hint + "\n" + long_task_prompt_example
    input_text = add_line_num(data["text"])
    messages = [
        {"role":"system","content":task_prompt},
        {"role":"user","content":f"input paragraph:\n{input_text}"}
    ]
    return messages

In [ ]:
async def getTranslation(item):
    if "messages" not in item:
        return None
    else:
        for i in range(3):
            result = await dealing_messages(item['messages'])
            if result is not None:
                item["response"] = result
                return item
            else:
                continue

    return None

async def process(item, semaphore):
    async with semaphore:
        try:
            output_folder = "/content/output"
            if "output_folder" in item:
                output_folder = item["output_folder"]
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)

            file_path = os.path.join(output_folder, f"{item['id']}.txt")

            if os.path.exists(file_path):
                return

            it = await getTranslation(item)
            if it is None:
                raise Exception(item['id'])

            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump(it, f, ensure_ascii=False)

        except Exception as e:
            print(f"Error processing entry: {e}")

In [ ]:
!rm -rf /content/output

In [ ]:
print(len(input_datas[0]['text']))

5732


In [ ]:
async def main(datas):

    output_folder = "/content/output"

    process_data = []

    for data in datas:
        id = data['id']
        process_data.append({
            "id": id,
            "messages": data2messages(data),
            "output_folder": output_folder
        })

    tasks = []

    semaphore = asyncio.Semaphore(concurrency_limit)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # # print(f"Already processed {len(exist_list)} items...")

    # id = set()

    for item in process_data:
        file_path = os.path.join(output_folder, f"{item['id']}.txt")

        if os.path.exists(file_path):
            continue

        tasks.append(asyncio.create_task(process(item, semaphore)))

    async for task in tqdm(tasks, total=len(tasks), desc="Processing items"):
        await task
        time.sleep(delay)



In [ ]:
!rm -rf /content/output

In [ ]:
!ls /content/drive/MyDrive/CardBuild/dialogue_extract/0122english_*

/content/drive/MyDrive/CardBuild/dialogue_extract/0122english_0_to_10.txt


In [ ]:
print(len(input_datas))

15804


In [ ]:
print(input_datas[0])

{'id': '0_second', 'text': 'Cory Doctorow wrote Little Brother in a white-hot fury over the course of eight weeks. The book was trying to get out of his head, leading to sleep deprivation and missed meals. The author\'s father was one of the few \'counterculture\' people who thought computers were a good thing in the 1960s. The author\'s fascination with activist causes grew as he saw how communications tools were being used to bring information and revolution to authoritarian states. However, 17 years later, computers are being used to spy, control, and restrict freedoms. The author expresses concern about the dangers of technology and the loss of privacy and freedom. The book is meant to be part of the conversation about what an information society means and encourages taking action for freedom and privacy.\nI wrote Little Brother in a white-hot fury between May 7, 2007 and July 2, 2007: exactly eight weeks from the day I thought it up to the day I finished it (Alice, to whom this bo

In [ ]:
datas = input_datas

In [ ]:
start_id = 0
end_id = 10

current_tasks = input_datas[start_id:end_id]

await main(current_tasks)

temp_output_folder = "/content/output"

for id in range(start_id, end_id):
    id_str = input_datas[id]["id"]
    file_path = os.path.join(temp_output_folder, f"{id_str}.txt")
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                response = data["response"]
            except:
                continue

        if response is not None:
            datas[id]["response"] = response
    # break

final_save_name = "/content/drive/MyDrive/CardBuild/exp0122/first/0123_eng_2nd_" + str(start_id) + "_to_" + str(end_id) + ".txt"

with open(final_save_name, 'w', encoding='utf-8') as f:
    for id in range(start_id, end_id):
        json.dump(datas[id], f, ensure_ascii=False)
        f.write('\n')

Processing items: 0it [00:00, ?it/s]


In [ ]:
datas = input_datas

In [ ]:
super_start = 10
super_end = len(datas)

n_bag = 40

# 计算每个子区间的长度
interval_length = (super_end - super_start) // n_bag

# 生成元组列表
se_tuples_list = [(super_start + i * interval_length, super_start + (i + 1) * interval_length) for i in range(n_bag - 1)]

# 添加最后一个元组，确保最后一个end等于super_end
se_tuples_list.append((se_tuples_list[-1][1], super_end))

# 打印生成的元组列表
print(se_tuples_list[0:2])
print(se_tuples_list[-2:])

import random


random.shuffle(se_tuples_list)

print(se_tuples_list[0])

[(10, 404), (404, 798)]
[(14982, 15376), (15376, 15804)]
(6314, 6708)


In [ ]:
from IPython.display import clear_output

for start_id, end_id in se_tuples_list:
    current_tasks = datas[start_id:end_id]

    await main(current_tasks)
    await main(current_tasks)

    temp_output_folder = "/content/output"

    for id in range(start_id, end_id):
        id_str = input_datas[id]["id"]
        file_path = os.path.join(temp_output_folder, f"{id_str}.txt")
        if os.path.exists(file_path):
            with open(file_path, 'r', encoding='utf-8') as f:
                try:
                    data = json.load(f)
                    response = data["response"]
                except:
                    continue

            if response is not None:
                datas[id]["response"] = response
        # break

    final_save_name = "/content/drive/MyDrive/CardBuild/exp0122/first/0123_eng_2nd_" + str(start_id) + "_to_" + str(end_id) + ".txt"

    with open(final_save_name, 'w', encoding='utf-8') as f:
        for id in range(start_id, end_id):
            json.dump(datas[id], f, ensure_ascii=False)
            f.write('\n')

Processing items: 100%|██████████| 394/394 [10:22<00:00,  1.58s/it]
Processing items: 0it [00:00, ?it/s]
Processing items: 100%|██████████| 394/394 [09:13<00:00,  1.40s/it]
Processing items: 0it [00:00, ?it/s]
Processing items: 100%|██████████| 394/394 [09:30<00:00,  1.45s/it]
Processing items: 0it [00:00, ?it/s]
Processing items: 100%|██████████| 394/394 [09:46<00:00,  1.49s/it]
Processing items: 0it [00:00, ?it/s]
Processing items: 100%|██████████| 394/394 [09:06<00:00,  1.39s/it]
Processing items: 0it [00:00, ?it/s]
Processing items: 100%|██████████| 394/394 [08:47<00:00,  1.34s/it]
Processing items: 0it [00:00, ?it/s]
Processing items: 100%|██████████| 394/394 [09:36<00:00,  1.46s/it]
Processing items: 0it [00:00, ?it/s]
Processing items: 100%|██████████| 394/394 [09:18<00:00,  1.42s/it]
Processing items: 0it [00:00, ?it/s]
Processing items: 100%|██████████| 394/394 [09:04<00:00,  1.38s/it]
Processing items: 0it [00:00, ?it/s]
Processing items: 100%|██████████| 394/394 [09:31<00:00